# EEP153 Project2

In [1]:
#!pip install -r requirements.txt

In [2]:
user = "ligon"

# API key for Gov; substitute your own!
apikey = "inIyO1begWSRqsYtxS7m6p09PSyq7Qiw7fxzV2qN"

# File with private keys for relevant service account to authenticate
# and access google spreadsheets
serviceacct = {'ligon':'./students-9093fa174318.json'}

In [3]:
SHEETs = [# Stigler's foods, modern prices
          ("https://docs.google.com/spreadsheet/ccc?key=1ObK5N_5aVXzVHE7ZXWBg0kQvPS3k1enRwsUjhytwh5A","Table B"), 
         ]

In [4]:
#### Need private keys from json file (we're authenticating using "service accounts")
!gpg --batch --passphrase "noodle octopus" -d ./students-9093fa174318.json.gpg > ./students-9093fa174318.json

gpg: AES256.CFB encrypted data
gpg: encrypted with 1 passphrase


In [5]:
import pandas as pd
from eep153_tools import read_sheets
df = pd.concat([read_sheets(spreadsheet,json_creds=serviceacct[user],sheet=sheet) for spreadsheet, sheet in SHEETs],
               ignore_index=True,sort=False)

### 1.2 Dietary Reference Intakes

In [6]:
DRIs = "https://docs.google.com/spreadsheets/d/1y95IsQ4HKspPW3HHDtH7QMtlDA66IUsCHJLutVL-MMc/"

# Define *minimums*
diet_min = read_sheets(DRIs,json_creds=serviceacct[user],sheet='diet_minimums').set_index('Nutrition')
# Define *maximums*
diet_max = read_sheets(DRIs,json_creds=serviceacct[user],sheet='diet_maximums').set_index('Nutrition')

In [9]:
from  scipy.optimize import linprog as lp
import numpy as np

def solve_subsistence_problem(FoodNutrients,Prices,diet_min,diet_max,tol=1e-6):
    p = Prices.apply(lambda x:x.magnitude).dropna()
    use = list(set(p.index.tolist()).intersection(FoodNutrients.columns.tolist()))
    p = p[use]
    Aall = FoodNutrients[p.index].fillna(0)
    Amin = Aall.loc[diet_min.index]
    Amax = Aall.loc[diet_max.index]
    A = pd.concat([Amin,-Amax])
    b = pd.concat([diet_min,-diet_max]) 
    result = lp(p, -A, -b, method='interior-point')
    result.A = A
    result.b = b
    result.diet = pd.Series(result.x,index=p.index)
    return result

### 1.8 Sensitivity of Solution

In [11]:
import fooddatacentral as fdc

D = {}
count = 0
for food in  df.Food.tolist():
    try:
        FDC = df.loc[df.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

FoodNutrients = pd.DataFrame(D,dtype=float)

In [14]:
# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
df['FDC Price'] = df['Price']/df['FDC Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food',sort=False)['FDC Price'].min()

In [16]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for M 19-30 is $7.03 per day.


Diet (in 100s of grams or milliliters):
Food
Wheat Cereal    0.313215
Cabbage         0.997122
Sugar           0.455579
Navy Beans      0.539075
Liver (Pork)    0.902305
Milk (Whole)    1.628001
Spinach         7.315484
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2400.000003          2400.0
Protein                           60.402307            56.0
Fiber, total dietary              33.600000            33.6
Folate, DFE                     1653.368775           400.0
Calcium, Ca                     1000.000000          1000.0
Carbohydrate, by difference      148.346580           130.0
Iron, Fe                          47.590071             8.0
Magnesium, Mg                    624.609875           400.0
Niacin                            25.136346            16.0
Phosp

/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



In [17]:
import cufflinks as cf
cf.go_offline()

scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]

cost0 = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

Price_response={}
for s in scale:
    cost = {}
    for i,p in enumerate(Prices):
        my_p = Prices.copy()
        my_p[i] = p*s
        result = solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
        cost[Prices.index[i]] = np.log(result.fun/cost0)
    Price_response[np.log(s)] = cost

Price_response = pd.DataFrame(Price_response).T
Price_response.iplot(xTitle='log price change',yTitle='log cost change')

/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray

